In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

In [46]:
# Custom Attention Module
class SimpleCustomAttention(nn.Module):
    def __init__(self, dim, num_heads):
        super(SimpleCustomAttention, self).__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.scaling = (dim // num_heads) ** -0.5

        self.qkv_proj = nn.Linear(dim, dim * 3)
        self.out_proj = nn.Linear(dim, dim)

    def forward(self, x, mask=None):
        batch_size, seq_length, dim = x.shape
        qkv = self.qkv_proj(x)
        q, k, v = torch.chunk(qkv, 3, dim=-1)
        q = q * self.scaling

        q = q.view(batch_size, seq_length, self.num_heads, self.dim // self.num_heads).transpose(1, 2)
        k = k.view(batch_size, seq_length, self.num_heads, self.dim // self.num_heads).transpose(1, 2)
        v = v.view(batch_size, seq_length, self.num_heads, self.dim // self.num_heads).transpose(1, 2)

        attn_weights = torch.matmul(q, k.transpose(-2, -1))
        if mask is not None:
            attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))

        attn_weights = F.softmax(attn_weights, dim=-1)

        output = torch.matmul(attn_weights, v)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_length, dim)
        output = self.out_proj(output)

        return output

In [47]:
# Custom Model incorporating the attention mechanism
class CustomSequenceClassifier(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super(CustomSequenceClassifier, self).__init__()
        self.attention = SimpleCustomAttention(hidden_dim, num_heads)
        self.classifier = nn.Linear(hidden_dim, 2)  # Binary classification

    def forward(self, x):
        attention_output = self.attention(x)
        pooled_output = attention_output.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits

class CustomSequenceClassifier(nn.Module):
    def __init__(self, hidden_dim, num_heads, vocab_size, embedding_dim):
        super(CustomSequenceClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.attention = SimpleCustomAttention(embedding_dim, num_heads)
        self.classifier = nn.Linear(embedding_dim, 2)  # Binary classification

    def forward(self, input_ids):
        x = self.embedding(input_ids)  # Embed input tokens
        attention_output = self.attention(x)
        pooled_output = attention_output.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits


In [15]:
!pip install transformers[torch]

In [16]:
!pip install accelerate -U

In [48]:
# Load dataset
file_path = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.xls'
data = pd.read_excel(file_path)
df = data[['sent', 'ManualLabel', 'codes']]
df_filtered = df[df['codes'].str.contains('Compatibility', case=False)].copy()
df_filtered.loc[:, 'ManualLabel'] = df_filtered['ManualLabel'].apply(lambda x: 1 if x == 'p' else 0)

# Split the dataset
train, test = train_test_split(df_filtered, test_size=0.4, random_state=42)

# Define dataset and loader
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Dummy tokenization and model setup (replace with appropriate tokenizer)
train_encodings = {'input_ids': np.random.randint(0, 2000, (len(train), 512))}
test_encodings = {'input_ids': np.random.randint(0, 2000, (len(test), 512))}
train_dataset = SentimentDataset(train_encodings, train['ManualLabel'].tolist())
test_dataset = SentimentDataset(test_encodings, test['ManualLabel'].tolist())

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Model and optimizer
vocab_size = 2001  # Update as necessary
embedding_dim = 768
model = CustomSequenceClassifier(hidden_dim=embedding_dim, num_heads=12, vocab_size=vocab_size, embedding_dim=embedding_dim)
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(10):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        logits = model(input_ids)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss {loss.item()}")



Epoch 1: Loss 0.6804978251457214
Epoch 2: Loss 0.6549054384231567
Epoch 3: Loss 0.6730349659919739
Epoch 4: Loss 0.63178551197052
Epoch 5: Loss 0.7214072346687317
Epoch 6: Loss 0.6474800705909729
Epoch 7: Loss 0.6010340452194214
Epoch 8: Loss 0.670124351978302
Epoch 9: Loss 0.6926778554916382
Epoch 10: Loss 0.6425260901451111


In [49]:
import time
from sklearn.metrics import accuracy_score, f1_score

# Evaluation loop
model.eval()
predictions, true_labels = [], []
total_inference_time = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)

        start_time = time.time()
        logits = model(input_ids)  # Directly output logits, no attention_mask needed
        inference_time = time.time() - start_time
        total_inference_time += inference_time
        total_samples += input_ids.size(0)

        preds = logits.argmax(dim=1).cpu().numpy()  # Extract predictions
        labels = batch['labels'].cpu().numpy()  # Extract true labels

        predictions.extend(preds)
        true_labels.extend(labels)

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
f1_micro = f1_score(true_labels, predictions, average='micro')
f1_macro = f1_score(true_labels, predictions, average='macro')
f1_weighted = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score (Micro): {f1_micro:.4f}')
print(f'F1 Score (Macro): {f1_macro:.4f}')
print(f'F1 Score (Weighted): {f1_weighted:.4f}')

# Calculate average inference time per sample
average_inference_time = total_inference_time / total_samples
print(f'Total Inference Time: {total_inference_time:.6f} seconds')
print(f'Total Samples: {total_samples}')
print(f'Average Inference Time per Sample: {average_inference_time:.6f} seconds')


Accuracy: 0.6842
F1 Score (Micro): 0.6842
F1 Score (Macro): 0.4063
F1 Score (Weighted): 0.5559
Total Inference Time: 3.976075 seconds
Total Samples: 38
Average Inference Time per Sample: 0.104634 seconds
